<a name="top"> <h1>01. ML Models</h1> <a>

<p>Análisis de sentimiento: Tweets<br />
<strong>Trabajo de Fin de Master</strong><br />
<strong>Master Universitario en Ciencia de Datos</strong></p>

<p>&nbsp;</p>

<p style="text-align:right">V&iacute;ctor Viloria V&aacute;zquez (<em>victor.viloria@cunef.edu</em>)</p>


<hr style="border:1px solid gray">

### Estructura

[1. Librerias utilizadas y funciones](#librerias) 

[2. Introducción ](#introduccion) 

   - Objetivo de negocio.

[3. Yelp Dataset ](#yelp) 

   - Información del dataset
   - Características del dataset


[4. Transformación del formato de ficheros](#transformacion) 


[5. Transformación de datos](#datos)

   - Business
       - Carga del fichero
       - Transformación de los datos
       - Exportación de ficheros procesados

<hr style="border:1px solid gray">

# <a name="librerias"> 1. Librerias utilizadas y funciones <a>


Importamos las librerias a utilizar para el preprocesamiento:

In [11]:
# Import libraries.

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from lazypredict.Supervised import LazyClassifier
import numpy as np
import string

# <a name="lectura"> 2. Lectura del dataframe <a>


In [18]:
#Import parquet file.

tweets = pd.read_parquet('../data/processed/tweets.parquet')

# Show the head of the dataframe.

tweets.head()

,text,sentiment,SentimentText_clean
0,id have responded if i were going,0,id have responded if i were going
1,sooo sad i will miss you here in san diego,2,sooo sad i will miss you here in san diego
2,my boss is bullying me,2,my boss is bullying me
3,what interview leave me alone,2,what interview leave me alone
4,sons of why couldnt they put them on the rel...,2,sons of why couldnt they put them on the rele...


In [21]:
X = tweets.SentimentText_clean
y = tweets.sentiment

In [26]:
# Split into train and test
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

In [30]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline

In [31]:

tfidf = TfidfVectorizer(stop_words="english",max_df=0.99,min_df=0.01)
svc = SVC()
model = make_pipeline(tfidf, svc)
#Training
model.fit(x_train, y_train)

#Prediccion sobre test
preds= model.predict(x_test )


In [33]:
np.mean(preds==y_test)

0.5852283063489175

In [37]:
# import multionomial naive bayes model

from sklearn.naive_bayes import MultinomialNB

# Entrenar el modelo
clf = MultinomialNB()

model = make_pipeline(tfidf, clf)
#Training
model.fit(x_train, y_train)

#Prediccion sobre test
preds= model.predict(x_test )


In [38]:
np.mean(preds==y_test)

0.5752228488266327

In [39]:
# Entrene con modelo xgboost

from xgboost import XGBClassifier

# Entrenar el modelo

xgb = XGBClassifier()

model = make_pipeline(tfidf, xgb)

#Training

model.fit(x_train, y_train)

#Prediccion sobre test

preds= model.predict(x_test )

np.mean(preds==y_test)

0.5834091322539567

In [40]:
# Entrene con modelo random forest

from sklearn.ensemble import RandomForestClassifier

# Entrenar el modelo

rf = RandomForestClassifier()

model = make_pipeline(tfidf, rf)

#Training

model.fit(x_train, y_train)

#Prediccion sobre test

preds= model.predict(x_test )

np.mean(preds==y_test)

0.5723121702746953

In [43]:
# Train with logistic regression

from sklearn.linear_model import LogisticRegression

# Entrenar el modelo

lr = LogisticRegression()

model = make_pipeline(tfidf, lr)

#Training

model.fit(x_train, y_train)

#Prediccion sobre test

preds= model.predict(x_test )

np.mean(preds==y_test)



0.585955975986902